In [ ]:
PARTITION_STR = "dt='20210721'"
DATASET = "di_mno_offer"
TABLE = "covid_social_contact_index_pred_ldong_cd_10_daily"

In [ ]:
from pydatafabric.gcp import bq_insert_overwrite
from pydatafabric.gcp import df_to_bq_table
from pydatafabric.ye import get_spark

In [ ]:
spark = get_spark()

In [ ]:
df = spark.sql(f"""
    select *
    from saturn.{TABLE} 
    where {PARTITION_STR}
""")
df_to_bq_table(df, "temp_1d", TABLE)

In [ ]:
bq_insert_overwrite(f"""
select * except(dt), parse_date('%Y%m%d', dt) as dt 
from temp_1d.{TABLE}
""", f"emart-datafabric.{DATASET}.{TABLE}")